In [3]:
import numpy as np
import pandas as pd

In [4]:
df_repairs = pd.read_csv("data/historic-repairs-2014-2015-cleaned.csv",
                         encoding="utf-8", index_col=False, parse_dates=['logged-date'])

In [5]:
#df_repairs.head(3)

### Let's remove columns I don't think will be useful

In [6]:
df_repairs_minimal = df_repairs.drop(['target-end-date', 'workforce-name', 'termination-date', 
                                      'original-target-date', 'priority-code'], 
                                     axis=1)

In [7]:
#df_repairs_minimal.head(3)

In [8]:
df_repairs_minimal.to_csv('data/historical-repairs-2014-2015-minimal.csv', index=False)

### Split dataset by year

In [9]:
#df_repairs_minimal[(df_repairs_minimal['logged-date'] <= '2012-12-31')]

#### There's only one repair event for 2012 so I want to delete it!

In [10]:
df_repairs_mini_after2012 = df_repairs_minimal.drop(df_repairs_minimal.index[[28649]])

In [11]:
df_repairs_mini_after2012[(df_repairs_mini_after2012['logged-date'] <= '2012-12-31')]

,logged-date,repair-number,description-for-code,property-reference


#### Let's look up the repairs in 2013:

In [12]:
df_repairs_2013 = df_repairs_mini_after2012[(df_repairs_mini_after2012['logged-date'] <= '2013-12-31')]

print "There were {0} repairs in 2013".format(len(df_repairs_2013))

There were 23 repairs in 2013


#### I feel this is incomplete. Also the dataset name mention "2014-2015" so I should focus on those years!

In [13]:
# I'm looking for the index of rows with repairs in 2013
df_repairs_2013.index

Int64Index([ 2257,  5235,  5497, 10265, 14385, 15774, 26723, 30625, 31440,
            31554, 31788, 32588, 33177, 34472, 35055, 35652, 41397, 42468,
            42702, 44908, 44938, 45171, 59910],
           dtype='int64')

In [14]:
df_repairs_mini_2014_2015 = df_repairs_mini_after2012.drop(df_repairs_2013.index)

In [15]:
df_repairs_mini_2014_2015[(df_repairs_mini_2014_2015['logged-date'] <= '2013-12-31')]

,logged-date,repair-number,description-for-code,property-reference


In [16]:
df_repairs_mini_2014_2015.to_csv('data/historical-repairs-2014-2015-mini-cleaned.csv', index=False)

#### Create the repairs 2014 dataset

In [17]:
df_repairs_mini_2014 = df_repairs_mini_2014_2015[(df_repairs_mini_2014_2015['logged-date'] <= '2014-12-31')]

In [18]:
#df_repairs_mini_2014.head()

In [19]:
#df_repairs_mini_2014.to_csv('data/historical-repairs-2014-mini.csv', index=False) #Incomplete

#### Create the repairs 2014 dataset

In [20]:
df_repairs_mini_2015 = df_repairs_mini_2014_2015[(df_repairs_mini_2014_2015['logged-date'] > '2014-12-31')]

In [21]:
#df_repairs_mini_2015.head()

In [22]:
#df_repairs_mini_2015.to_csv('data/historical-repairs-2015-mini.csv', index=False) #Incomplete

### Handle "2016" repairs data

In [23]:
df_repairs_2016 = pd.read_csv("data/historic-repairs-data-2016.csv",
                              encoding="utf-8", index_col=False, parse_dates=['Logged date'])

In [24]:
#df_repairs_2016.head(3)

In [25]:
df_2016_minimal = df_repairs_2016.drop(['Current target', 'Workforce name', 'Termination date', 
                                        'Original target', 'Priority', 'Cost code',
                                        'Category of provision summary', 'Estate code'], 
                                       axis=1)

In [26]:
#df_2016_minimal.head(3)

In [27]:
df_2016_minimal.to_csv('data/historical-repairs-2016-minimal.csv', index=False)

In [28]:
df_extra_2014 = df_2016_minimal[(df_2016_minimal['Logged date'] <= '2014-12-31')]

#df_extra_2014

In [29]:
df_extra_2014 = df_extra_2014.rename(columns={'Repair number': 'repair-number',
                                              'Property reference': 'property-reference',
                                              'Logged date': 'logged-date',
                                              'Trade name': 'description-for-code'})

In [30]:
print len(df_repairs_mini_2014) + len(df_extra_2014)

df_full_2014_repairs = pd.concat([df_repairs_mini_2014, df_extra_2014])

print len(df_full_2014_repairs)

46798
46798


In [31]:
df_full_2014_repairs.to_csv("data/historical-repairs-2014-merge.csv", index=False)

In [32]:
df_extra_2015_2016 = df_2016_minimal.drop(df_extra_2014.index)

In [33]:
df_extra_2015 = df_extra_2015_2016[(df_extra_2015_2016['Logged date'] <= '2015-12-31')]

#df_extra_2015.head(3)

In [34]:
df_extra_2015 = df_extra_2015.rename(columns={'Repair number': 'repair-number',
                                              'Property reference': 'property-reference',
                                              'Logged date': 'logged-date',
                                              'Trade name': 'description-for-code'})

In [35]:
print len(df_repairs_mini_2015) + len(df_extra_2015)

df_full_2015_repairs = pd.concat([df_repairs_mini_2015, df_extra_2015])

print len(df_full_2015_repairs)

64858
64858


In [36]:
df_full_2015_repairs.to_csv("data/historical-repairs-2015-merge.csv", index=False)

In [37]:
df_repairs_2016 = df_extra_2015_2016[(df_extra_2015_2016['Logged date'] >= '2016-01-01')]

#df_repairs_2016.head()

In [38]:
df_repairs_2016 = df_repairs_2016.rename(columns={'Repair number': 'repair-number',
                                                  'Property reference': 'property-reference',
                                                  'Logged date': 'logged-date',
                                                  'Trade name': 'description-for-code'})

In [39]:
df_repairs_2016.to_csv("data/historical-repairs-2016-mini.csv", index=False)

### Bring together all repair data (2014, 2015, 2016)

In [40]:
df_all_repairs = pd.concat([df_full_2014_repairs, df_full_2015_repairs, df_repairs_2016])

In [41]:
print len(df_full_2014_repairs) + len(df_full_2015_repairs) + len(df_repairs_2016),len(df_all_repairs)

#df_all_repairs.head(3)

131750 131750


In [42]:
df_all_repairs.to_csv("data/all-historical-repairs-mini.csv", index=False)

### Prepare repair costs data

In [43]:
df_costs = pd.read_csv("data/costs-historic-repairs-2014-2015.csv",
                       encoding="utf-8", index_col=False, parse_dates=['Logged date'])

In [44]:
#df_costs.head(3)

In [45]:
df_costs_minimal = df_costs.drop(['Current target', 'Workforce name', 'Termination date', 
                                  'Original target', 'Priority', 'Cost code',
                                  'Category of provision summary'], 
                                 axis=1)

In [46]:
#df_costs_minimal.head(3)

In [47]:
df_costs_minimal.to_csv('data/costs-historical-repairs-2014-2015-minimal.csv', index=False)